In [ ]:
import tweepy
import csv
from time import gmtime, strftime
import random
import nltk
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [ ]:
#Twitter API credentials
consumer_key = "Insert your consumer key"
consumer_secret = "Insert your consumer secret"
access_key = "Insert your access key"
access_secret = "Insert your access secret"

In [ ]:
# Initiate the groups/keywords that you want the streamer to search for
#The code is written in such a way that you only need to change the value of these
#   variables when you are changing the keywords
group1 = 'Saints'
group2 = 'Falcons'

# All tweets will search for a lower-case match, so I ensure that each group is lower-case upon initiated them
group1 = group1.lower()
group2 = group2.lower()

In [ ]:
#Authorize Twitter/Initialize Tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [ ]:
# Initiate lists and running total used for sentiment
group1Scores = []
group1TotalScore = 0
group2Scores = []
group2TotalScore = 0


class MyStreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        # Create Dictionary and bring in the lists as global variables
        sentiment_scores = {}
        global group1Scores
        global group1TotalScore
        global group2Scores
        global group2TotalScore
        print(status)
        # Open the file `tidytext_sentiments.txt`
        with open('/Users/Kelly/Documents/BMIS694/Live Tweet Sentiment/tidytext_sentiments.txt') as f:
            next(f)             
            for line in f:
                (word,sentiment,lexicon) = line.split('\t')
                if sentiment == "negative":
                    sentiment_scores[word] = -1
                if sentiment == "positive":
                    sentiment_scores[word] = 1
        try:
            # Assign tweets to variable and clean up tweets 
            tweet = status.text
            tweet = tweet.lower()
            tweet = tweet.replace('\n', ' ')
            tweet = tweet.replace('\t', ' ')
            
            # If both groups are mentioned in the tweet, then skip, because they will cancel each other out
            if (group1 in tweet) and (group2 in tweet): #or ('rt ' in tweet):
                #print("****"+tweet+"****")
                next
            
            # Loop for tweets that contain the group1 variable somewhere in the tweet
            elif group1 in tweet:
                #print(group1 + '\t' + tweet +'\n')         #Uncomment this out to ensure tweets are streaming as expected
                tweet = tweet.split()           # Split the tweets into individual word
                group1TweetScores = []          # Create an empty list for the sentiment scores of each individual tweet
                for word in tweet:

                    if word in sentiment_scores:
                        group1TotalScore = (sentiment_scores[word] + group1TotalScore)
                        group1TweetScores.extend([group1TotalScore]) 

    #                     if sentiment_scores[word] == -1:
    #                         print(word +' - negative')
    #                     if sentiment_scores[word] == 1:
    #                         print(word +' - positive')
                        group1Scores= []
                        group1Scores = [group1TotalScore]
                    else:
                        group1TweetScores.extend([group1TotalScore])
                        group1Scores= []
                        group1Scores = [group1TotalScore]
                with open('/Users/Kelly/Documents/BMIS694/Live Tweet Sentiment/Sentiment Files/'+group1 + group2 +'Sentiment.txt', 'a') as f:
                    for idx, score in enumerate(group1TweetScores):
                        f.write(strftime(group1 + '\t' +"%H:%M") + '\t' + str(score) + '\n')
                    f.close()
                
            elif group2 in tweet:
                #print(group2 + "\t" + tweet + '\n')
                tweet = tweet.split()
                group2TweetScores = []
                for word in tweet:

                    if word in sentiment_scores:
                        group2TotalScore = (sentiment_scores[word] + group2TotalScore)
                        group2TweetScores.extend([group2TotalScore]) 

    #                     if sentiment_scores[word] == -1:
    #                         print(word +' - negative')
    #                     if sentiment_scores[word] == 1:
    #                         print(word +' - positive')
                        group2Scores= []
                        group2Scores = [group2TotalScore]
                    else:
                        group2TweetScores.extend([group2TotalScore])
                        group2Scores= []
                        group2Scores = [group2TotalScore]
                with open('/Users/Kelly/Documents/BMIS694/Live Tweet Sentiment/Sentiment Files/'+ group1 + group2 +'Sentiment.txt', 'a') as f:
                    for idx, score in enumerate(group2TweetScores):
                        f.write(strftime(group2 + '\t' + "%H:%M") + '\t' + str(score) + '\n')
                        
                    f.close()
                
            else:
                next
            return True
        except BaseException as e:
            print('****************************failed on_status,',str(e)+'****************************')
            time.sleep(5)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, wait_on_rate_limit = True)

In [ ]:
myStream.filter(track=[group1, group2])